In [1]:
import pysam
from pysam import AlignmentFile
from collections import defaultdict

## Indicizzare e aprire il file

In [2]:
pysam.index('./reads.aln.bam')
bam_file = AlignmentFile('./reads.aln.bam', 'rb')

## Aprire il reference

In [3]:
ref = ''
for line in open('ref.fa', 'r'):
    if not line.startswith(">"):
        line = line.strip()
        ref+=line.upper()
        
print(ref[:100])
print(len(ref))

GATCCACCCGCCTTGGCCTCCTAAAGTGCTGGGATTACAGGTGTTAGCCACCACGTCCAGCTGTTAATTTTTATTTAATAAGAATGACAGAGTGAGGGCC
4999950


## Usare `pileup` per trovare le posizioni che non concordano tra REFERENCE e READS

Attenzione a quante/quali posizioni alternative sono presenti nelle reads

In [4]:
for pileupcolumn in bam_file.pileup(min_base_quality = 0):
    refpos = ref[pileupcolumn.pos]
    bampos = defaultdict(int)
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            bampos[pileupread.alignment.query_sequence[pileupread.query_position]] += 1
    if len(bampos) > 0:
        support = sorted(bampos, key=bampos.get)
        if len(support) == 1:
            if support[0] != refpos:
                print(pileupcolumn.pos, refpos, support[0], sep="\t")
        else:
            if support[0] == refpos:
                print(pileupcolumn.pos, refpos, support[1], sep="\t")
            elif support[1] == refpos:
                print(pileupcolumn.pos, refpos, support[0], sep="\t")
            else:
                print(pileupcolumn.pos, refpos, f"{support[0]},{support[1]}", sep="\t")

18988	T	A
22824	C	T
23404	C	A
32063	C	T
32813	G	A
35347	C	G
36064	T	C
36091	G	A
36124	T	C
42334	T	G
42888	C	A
43252	A	C
48099	A	T
48712	G	T
49367	G	T
49819	C	A
50115	G	C
51122	G	A
51475	C	T
51515	T	C
56086	G	A
56166	C	G
59517	G	T
60022	A	T
61197	C	T
81336	G	T
81339	G	A
92804	C	T
93827	C	A
94382	A	T
96315	C	T
103015	G	T
103225	G	T
103731	C	G
106954	T	A
115047	C	T
131236	G	C
133897	C	T
146389	G	T
154426	G	C
154432	G	A
154443	T	G
154541	C	A
156011	T	C
156168	A	C
156214	C	A
156424	A	C
156701	G	T
161811	C	T
173481	A	C
174492	C	A
175428	G	T
182174	A	G
186472	C	T
189833	G	C
192782	T	G
193213	T	A
195979	G	C
196166	C	T
196169	T	G
196178	C	T
197008	C	T
197118	G	A
204968	G	C
205083	T	G
206043	T	A
206074	T	C
206107	G	A
206862	T	A
206891	G	A
218542	T	A
218699	C	G
218785	A	T
218995	T	C
221198	T	C
221233	T	A
222596	C	T
225737	G	A
227914	G	C
230881	T	A
230942	A	T
239607	C	G
239727	T	C
246485	T	C
254358	A	T
256047	A	C
256824	A	C
256840	C	A
256892	G	C
258168	T	G
258196	G	T
258229	T	A
261110	A	C
263551	C

## Aggiungere genotipo

- `0/0`: le reads supportano solo il REFERENCE (nel nostro caso non ci interessa)
- `0/1`: le reads supportano sia il REFERENCE che l'ALTERNATIVE
- `1/1`: le reads supportano solo l'ALTERNATIVE
- `1/2`: sono presenti 2 ALTERNATIVE e le reads supportano solo i due ALTERNATIVE

In [5]:
for pileupcolumn in bam_file.pileup(min_base_quality = 0):
    refpos = ref[pileupcolumn.pos]
    bampos = defaultdict(int)
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            bampos[pileupread.alignment.query_sequence[pileupread.query_position]] += 1
    if len(bampos) > 0:
        support = sorted(bampos, key=bampos.get)
        if len(support) == 1:
            if support[0] != refpos:
                print(pileupcolumn.pos, refpos, support[0], "1/1", sep="\t")
        else:
            if support[0] == refpos:
                print(pileupcolumn.pos, refpos, support[1], "0/1", sep="\t")
            elif support[1] == refpos:
                print(pileupcolumn.pos, refpos, support[0], "0/1", sep="\t")
            else:
                print(pileupcolumn.pos, refpos, f"{support[0]},{support[1]}", "1/2", sep="\t")

18988	T	A	0/1
22824	C	T	1/1
23404	C	A	1/1
32063	C	T	1/1
32813	G	A	0/1
35347	C	G	1/1
36064	T	C	1/1
36091	G	A	1/1
36124	T	C	1/1
42334	T	G	1/1
42888	C	A	1/1
43252	A	C	1/1
48099	A	T	1/1
48712	G	T	1/1
49367	G	T	0/1
49819	C	A	1/1
50115	G	C	0/1
51122	G	A	1/1
51475	C	T	1/1
51515	T	C	1/1
56086	G	A	1/1
56166	C	G	1/1
59517	G	T	1/1
60022	A	T	1/1
61197	C	T	1/1
81336	G	T	1/1
81339	G	A	1/1
92804	C	T	1/1
93827	C	A	1/1
94382	A	T	1/1
96315	C	T	1/1
103015	G	T	1/1
103225	G	T	0/1
103731	C	G	1/1
106954	T	A	0/1
115047	C	T	1/1
131236	G	C	1/1
133897	C	T	1/1
146389	G	T	1/1
154426	G	C	1/1
154432	G	A	1/1
154443	T	G	1/1
154541	C	A	1/1
156011	T	C	1/1
156168	A	C	0/1
156214	C	A	1/1
156424	A	C	1/1
156701	G	T	0/1
161811	C	T	1/1
173481	A	C	1/1
174492	C	A	1/1
175428	G	T	0/1
182174	A	G	1/1
186472	C	T	1/1
189833	G	C	1/1
192782	T	G	0/1
193213	T	A	0/1
195979	G	C	1/1
196166	C	T	1/1
196169	T	G	1/1
196178	C	T	1/1
197008	C	T	0/1
197118	G	A	0/1
204968	G	C	1/1
205083	T	G	1/1
206043	T	A	0/1
206074	T	C	0/1
206107	G	A	0/1
206862	T	A	

## Aggiungere campi `AD:DP` sul VCF

- `AD`: Allele Difference, per ogni allele rappresenta il totale di reads che lo supportano seperati da `,`
- `DP`: Depth Position, rappresenta la somma totale di reads che mappano in quella posizione

In [6]:
for pileupcolumn in bam_file.pileup(min_base_quality = 0):
    refpos = ref[pileupcolumn.pos]
    bampos = defaultdict(int)
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            bampos[pileupread.alignment.query_sequence[pileupread.query_position]] += 1
    if len(bampos) > 0:
        support = sorted(bampos, key=bampos.get)
        if len(support) == 1:
            if support[0] != refpos:
                print(pileupcolumn.pos, refpos, support[0], "1/1", "AD:DP", f"{0},{bampos[support[0]]}:{bampos[support[0]]}", sep="\t")
        else:
            if support[0] == refpos:
                print(pileupcolumn.pos, refpos, support[1], "0/1", "AD:DP", f"{bampos[support[0]]},{bampos[support[1]]}:{sum(bampos.values())}", sep="\t")
            elif support[1] == refpos:
                print(pileupcolumn.pos, refpos, support[0], "0/1", "AD:DP", f"{bampos[support[1]]},{bampos[support[0]]}:{sum(bampos.values())}", sep="\t")
            else:
                print(pileupcolumn.pos, refpos, f"{support[0]},{support[1]}", "1/2", "AD:DP", f"{bampos[support[0]]},{bampos[support[1]]}:{sum(bampos.values())}", sep="\t")

18988	T	A	0/1	AD:DP	1,1:2
22824	C	T	1/1	AD:DP	0,1:1
23404	C	A	1/1	AD:DP	0,1:1
32063	C	T	1/1	AD:DP	0,1:1
32813	G	A	0/1	AD:DP	1,1:2
35347	C	G	1/1	AD:DP	0,1:1
36064	T	C	1/1	AD:DP	0,1:1
36091	G	A	1/1	AD:DP	0,1:1
36124	T	C	1/1	AD:DP	0,1:1
42334	T	G	1/1	AD:DP	0,1:1
42888	C	A	1/1	AD:DP	0,1:1
43252	A	C	1/1	AD:DP	0,1:1
48099	A	T	1/1	AD:DP	0,1:1
48712	G	T	1/1	AD:DP	0,1:1
49367	G	T	0/1	AD:DP	1,1:2
49819	C	A	1/1	AD:DP	0,1:1
50115	G	C	0/1	AD:DP	1,1:2
51122	G	A	1/1	AD:DP	0,1:1
51475	C	T	1/1	AD:DP	0,1:1
51515	T	C	1/1	AD:DP	0,1:1
56086	G	A	1/1	AD:DP	0,1:1
56166	C	G	1/1	AD:DP	0,1:1
59517	G	T	1/1	AD:DP	0,1:1
60022	A	T	1/1	AD:DP	0,1:1
61197	C	T	1/1	AD:DP	0,1:1
81336	G	T	1/1	AD:DP	0,1:1
81339	G	A	1/1	AD:DP	0,1:1
92804	C	T	1/1	AD:DP	0,1:1
93827	C	A	1/1	AD:DP	0,1:1
94382	A	T	1/1	AD:DP	0,1:1
96315	C	T	1/1	AD:DP	0,1:1
103015	G	T	1/1	AD:DP	0,1:1
103225	G	T	0/1	AD:DP	1,1:2
103731	C	G	1/1	AD:DP	0,1:1
106954	T	A	0/1	AD:DP	1,1:2
115047	C	T	1/1	AD:DP	0,1:1
131236	G	C	1/1	AD:DP	0,1:1
133897	C	T	1/1	AD:DP	0,1:1
14638